#Algoritmos - Actividad 3

Nombre: William Vasquez
<br>
URL: https://github.com/williamvp10/Algoritmos_optimizacion_UIV/blob/main/G3/Algoritmo_Genetico_para_el_TSP.ipynb

#Instalación de librerias

In [81]:
!pip install requests
import urllib.request

!pip install tsplib95
import tsplib95

In [82]:
import random   #Libreria para generar numeros y listas aleatorias
import copy     #Permite hacer copias de objetos en python: listas, diccionarios,...

#Carga de datos del problema

In [83]:
#Librerias y carga del problema

#http://elib.zib.de/pub/mp-testdata/tsp/tsplib/
#Documentacion :
  # https://wwwproxy.iwr.uni-heidelberg.de/groups/comopt/software/TSPLIB95/tsp95.pdf
  # https://tsplib95.readthedocs.io/usage.html
  # https://tsplib95.readthedocs.io/modules.html#module-tsplib95.models

#Matriz de distancias
file = "swiss42.tsp" ;
urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz", file + '.gz')
!gzip -d swiss42.tsp.gz     #Descomprimir el fichero de datos

#Objeto de tsplib95 para nuestro problema problema
problem = tsplib95.load(file)

#Nodos
Nodos = list(problem.get_nodes())

#Aristas
Aristas = list(problem.get_edges())
print(Aristas)

#Coordenadas(si estan disponibles en el ficher)
problem.get_display(1)

#Distancia
problem.get_weight(1, 2)



[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (0, 11), (0, 12), (0, 13), (0, 14), (0, 15), (0, 16), (0, 17), (0, 18), (0, 19), (0, 20), (0, 21), (0, 22), (0, 23), (0, 24), (0, 25), (0, 26), (0, 27), (0, 28), (0, 29), (0, 30), (0, 31), (0, 32), (0, 33), (0, 34), (0, 35), (0, 36), (0, 37), (0, 38), (0, 39), (0, 40), (0, 41), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (1, 14), (1, 15), (1, 16), (1, 17), (1, 18), (1, 19), (1, 20), (1, 21), (1, 22), (1, 23), (1, 24), (1, 25), (1, 26), (1, 27), (1, 28), (1, 29), (1, 30), (1, 31), (1, 32), (1, 33), (1, 34), (1, 35), (1, 36), (1, 37), (1, 38), (1, 39), (1, 40), (1, 41), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (2, 11), (2, 12), (2, 13), (2, 14), (2, 15), (2, 16), (2, 17), (2, 18), (2, 19), (2, 20), (2, 21), (2, 22), (2, 23), (2, 24), (2, 25), (2, 26), (2, 27), (2, 28), (2, 29), (2,

34

#Funciones de la Actividad Guiada 3

In [84]:
#Funciones de la Actividad Guiada 3
#Se genera una solucion aleatoria con comienzo en en el nodo 0
def crear_solucion(Nodos):
  solucion = [Nodos[0]]
  for n in Nodos[1:] :
    solucion = solucion + [random.choice(list(set(Nodos) - set(solucion)))]
  return solucion

#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
  return problem.get_weight(a,b)


#Devuelve la distancia total de una trayectoria/solucion
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)

#Funciones Auxiliares

In [85]:
#Genera una poblacion inicial de soluciones de tamaño N.
# Puede ser válida la solución aleatoria de la AG3: crear_solucion(Nodos)
def generar_poblacion(Nodos,N):
  poblacion = []
  for i in range(N):
    poblacion.append(crear_solucion(Nodos))
  return poblacion

In [86]:
#Evalua la población y devuelve el mejor individuo
def Evaluar_Poblacion(poblacion, problem):
  mejor_solucion = poblacion[0]
  mejor_distancia = distancia_total(poblacion[0], problem)
  
  for individuo in poblacion:
    distancia_actual = distancia_total(individuo, problem)
    if distancia_actual < mejor_distancia:
      mejor_solucion = individuo
      mejor_distancia = distancia_actual
  
  return (mejor_solucion, mejor_distancia)

In [87]:
#Funcion de cruce. Recibe una poblacion(lista de soluciones) y devuelve la población ampliada con los hijos.
# Todos los individuos de la población son selecionados para el cruce(si la población es par)
# Podría aplicarse un proceso previo de selección para elegir los individuos que se desea cruzar.
def Cruzar(poblacion,mutacion, problem):
  nueva_poblacion = copy.deepcopy(poblacion)
  
  # Mezclar la población para emparejar aleatoriamente
  random.shuffle(poblacion)
  
  # Cruzar de a pares
  for i in range(0, len(poblacion)-1, 2):
    padre1 = poblacion[i]
    padre2 = poblacion[i+1]
    
    # Generar descendencia
    hijos = Descendencia([padre1, padre2], problem, mutacion)
    
    # Agregar hijos a la nueva población
    nueva_poblacion.extend(hijos)
  
  return nueva_poblacion

In [88]:
#Funcion para generar hijos a partir de 2 padres:
# Se elige el metodo de 1-punto de corte pero es posible usar otros n-puntos, uniforme, dependiendo del problema
# MEJORA SIMPLE: Incluye una mejora local básica ocasional
def Descendencia(padres, problem,mutacion):
  padre1, padre2 = padres
  
  # Usar Order Crossover (OX) para TSP
  # Seleccionar dos puntos de corte aleatorios
  punto1 = random.randint(1, len(padre1)-2)
  punto2 = random.randint(punto1+1, len(padre1)-1)
  
  # Crear hijos
  hijo1 = [-1] * len(padre1)
  hijo2 = [-1] * len(padre2)
  
  # Copiar la sección entre los puntos de corte
  hijo1[punto1:punto2] = padre1[punto1:punto2]
  hijo2[punto1:punto2] = padre2[punto1:punto2]
  
  # Completar hijo1 con elementos de padre2
  hijo1 = Factibilizar(hijo1, padre2)
  
  # Completar hijo2 con elementos de padre1
  hijo2 = Factibilizar(hijo2, padre1)
  
  # Aplicar mutación
  hijo1 = Mutar(hijo1, mutacion)
  hijo2 = Mutar(hijo2, mutacion)
  
  # MEJORA SIMPLE: Ocasionalmente aplica una mejora local básica (2-opt simple)
  # Esto ayuda a entender por qué AG3 obtiene mejores resultados
  if random.random() < 0.3:  # 30% de probabilidad
    hijo1 = mejora_local_simple(hijo1, problem)
  
  if random.random() < 0.3:  # 30% de probabilidad  
    hijo2 = mejora_local_simple(hijo2, problem)
  
  return [hijo1, hijo2]

In [89]:
#Para el operador de cruce 1-punto los hijos generados no son soluciones(algunos nodos se repiten y otros no están)
def Factibilizar(hijo_parcial, padre_donante):
  # Completar el hijo con elementos del padre donante en orden
  elementos_faltantes = []
  
  # Encontrar elementos que faltan
  for elemento in padre_donante:
    if elemento not in hijo_parcial:
      elementos_faltantes.append(elemento)
  
  # Llenar las posiciones vacías (-1) con elementos faltantes
  indice_faltante = 0
  for i in range(len(hijo_parcial)):
    if hijo_parcial[i] == -1:
      hijo_parcial[i] = elementos_faltantes[indice_faltante]
      indice_faltante += 1
  
  return hijo_parcial

In [90]:
#Funcion de mutación. Se eligen dos nodos y se intercambia. Se podrian añadir otros operaradores
# Se hace mutaciones mutacion% de las veces
def Mutar(solucion,mutacion):
  if random.random() < mutacion:
    # Seleccionar dos posiciones aleatorias (evitando el primer nodo que es siempre 0)
    pos1 = random.randint(1, len(solucion)-1)
    pos2 = random.randint(1, len(solucion)-1)
    
    # Asegurar que las posiciones sean diferentes
    while pos1 == pos2:
      pos2 = random.randint(1, len(solucion)-1)
    
    # Intercambiar los elementos
    solucion[pos1], solucion[pos2] = solucion[pos2], solucion[pos1]
  
  return solucion

In [91]:
#Funcion de seleccion de la población. Recibe como parametro una poblacion y
# devuelve una poblacion a la que se ha eliminado individuos poco aptos(fitness alto) y para mantener una poblacion estable de N individuos
#Se tiene en cuenta el porcentaje elitismo pasado como parametro
# Para los individuos que no son de la elite podríamos usar una selección de ruleta(proporcional a su fitness)
def Seleccionar(problem,poblacion, N, elitismo):
  # Calcular fitness de toda la población
  fitness_poblacion = []
  for individuo in poblacion:
    fitness_poblacion.append((individuo, distancia_total(individuo, problem)))
  
  # Ordenar por fitness (distancia menor = mejor fitness)
  fitness_poblacion.sort(key=lambda x: x[1])
  
  # Calcular cuántos individuos van por elitismo
  num_elite = int(N * elitismo)
  
  # Seleccionar elite
  nueva_poblacion = []
  for i in range(num_elite):
    nueva_poblacion.append(fitness_poblacion[i][0])
  
  # Completar el resto con selección por torneo
  while len(nueva_poblacion) < N:
    # Selección por torneo (tamaño del torneo = 3)
    torneo = random.sample(fitness_poblacion, min(3, len(fitness_poblacion)))
    ganador = min(torneo, key=lambda x: x[1])
    nueva_poblacion.append(ganador[0])
  
  return nueva_poblacion


In [92]:
# MEJORA SIMPLE: Una función básica de mejora local 
# Esta es una versión simplificada de lo que hace AG3 mejor

def mejora_local_simple(solucion, problem):
  """
  Aplica una mejora local muy básica: intenta unas pocas inversiones de segmentos
  Esta es la razón principal por la que AG3 obtiene mejores resultados
  """
  mejor_solucion = solucion[:]
  mejor_distancia = distancia_total(mejor_solucion, problem)
  
  # Intentar solo unas pocas mejoras para no complicar mucho
  for _ in range(3):  # Solo 3 intentos
    # Seleccionar dos posiciones al azar
    i = random.randint(1, len(solucion) - 3)
    j = random.randint(i + 2, len(solucion) - 1)
    
    # Crear nueva solución invirtiendo el segmento entre i y j
    nueva_solucion = solucion[:]
    nueva_solucion[i:j+1] = reversed(nueva_solucion[i:j+1])
    
    # Si es mejor, la guardamos
    nueva_distancia = distancia_total(nueva_solucion, problem)
    if nueva_distancia < mejor_distancia:
      mejor_solucion = nueva_solucion
      mejor_distancia = nueva_distancia
  
  return mejor_solucion


#Proceso Principal

In [ ]:
#Funcion principal del algoritmo genetico
# MEJORA SIMPLE: Incluye ocasionalmente mejora local básica
def algoritmo_genetico(problem=problem,N=100,mutacion=.15,elitismo=.1,generaciones=100):
  # problem = datos del problema
  # N = Tamaño de la población
  # mutacion = probabilidad de una mutación
  # elitismo = porcion de la mejor poblacion a mantener
  # generaciones = nº de generaciones a generar para finalizar

  #Genera la poblacion inicial
  Nodos = list(problem.get_nodes())
  poblacion = generar_poblacion(Nodos,N)

  #Inicializamos valores para la mejor solucion
  (mejor_solucion, mejor_distancia) = Evaluar_Poblacion(poblacion, problem)

  #Condicion de parada
  parar = False
  n=0
  #Inciamos el cliclo de generaciones
  while(parar == False) :

    #Cruce de la poblacion(incluye mutación y mejora local básica)
    poblacion = Cruzar(poblacion, mutacion, problem)

    #Seleccionamos la población
    poblacion = Seleccionar(problem,poblacion, N, elitismo)

    #Evaluamos la nueva población
    (mejor_solucion, mejor_distancia) = Evaluar_Poblacion(poblacion, problem)

    print("Generacion #", n, "\nLa mejor solución es:" , mejor_solucion, "\ncon distancia " , mejor_distancia, "\n")

    #Numero de generaciones. Criterio de parada
    if n==generaciones:
      parar = True
    n +=1

  return mejor_solucion




In [ ]:
# Ejecutar el algoritmo genético con parámetros más pequeños para prueba
# sol = algoritmo_genetico(problem=problem,N=50,mutacion=.2,elitismo=.3,generaciones=10)

# Para ejecutar con parámetros más grandes:
# sol = algoritmo_genetico(problem=problem,N=500,mutacion=.3,elitismo=.40,generaciones=250)
# Ejecución del algoritmo genético mejorado para TSP
print("=" * 60)
print("ALGORITMO GENÉTICO PARA TSP")
print("=" * 60)
print("Número de nodos:", len(Nodos))
print("Parámetros: N=500, mutación=0.3, elitismo=0.40, generaciones=250")
print()

# Ejecutar algoritmo con mejora local básica
sol_final = algoritmo_genetico(problem=problem, N=500, mutacion=0.3, elitismo=0.40, generaciones=250)

print("=" * 60)
print("RESULTADO FINAL:")
print("Mejor solución encontrada:", sol_final)
print("Distancia total:", distancia_total(sol_final, problem))
print()

print("=" * 60)


ALGORITMO GENÉTICO PARA TSP
Número de nodos: 42
Parámetros: N=30, mutación=0.2, elitismo=0.3, generaciones=15

Generacion # 0 
La mejor solución es: [0, 34, 22, 24, 29, 30, 37, 6, 27, 13, 19, 14, 31, 3, 17, 16, 11, 23, 28, 18, 4, 7, 32, 20, 2, 1, 39, 8, 41, 36, 5, 9, 25, 38, 35, 33, 40, 10, 12, 21, 15, 26] 
con distancia  4065 

Generacion # 1 
La mejor solución es: [0, 18, 6, 3, 27, 21, 2, 35, 22, 10, 7, 5, 19, 13, 31, 30, 33, 37, 36, 12, 41, 29, 11, 1, 26, 25, 23, 16, 15, 32, 20, 34, 9, 39, 40, 24, 8, 4, 17, 38, 28, 14] 
con distancia  3825 

Generacion # 2 
La mejor solución es: [0, 36, 33, 3, 20, 1, 16, 23, 12, 10, 13, 6, 41, 39, 30, 29, 11, 5, 26, 32, 22, 21, 28, 8, 9, 37, 15, 27, 34, 2, 4, 19, 14, 35, 17, 25, 18, 38, 24, 40, 7, 31] 
con distancia  3747 

Generacion # 3 
La mejor solución es: [0, 34, 30, 20, 16, 19, 13, 29, 7, 26, 12, 3, 31, 33, 36, 14, 23, 10, 41, 6, 37, 15, 4, 5, 1, 22, 21, 28, 8, 9, 11, 25, 38, 18, 2, 35, 17, 32, 24, 40, 39, 27] 
con distancia  3573 

Generacio